<h3> Pre-processing steps </h3>
<ol> 
    <li> Add bool or int to: 
         <ul>
             <li> Tags </li>
             <li> Links </li>
             <li> KeywordNull </li>
         </ul>
    </li>
    <li> Remove symbols and numbers from: 
         <ul>
             <li> Keyword </li>
             <li> Tweets </li>
         </ul>
    </li>
    <li> Convert the given document to matrix: 
        <ol>
            <li> Convert the tweet to vector </li>
            <li> add new columns, which are </li>
        </ol>
            
</ol>

<h4>def addBool:</h4>
<h4>def removeSymbols:</h4>
<h4>def convertToMatrix:</h4>

In [2]:
import pandas as pd
import numpy as np
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import matplotlib.pyplot as plt
import copy

In [3]:
with open(r"\Documents\Data analytics\Project\Work\Notebook\Untitled Folder\words_alpha.txt") as wordFile:
    words = set(wordFile.read().split())

In [4]:
pathTest = r"\Documents\Data analytics\Project\Data\test.csv"
pathTrain = r"\Documents\Data analytics\Project\Data\train.csv"
test_df = pd.read_csv(pathTest)
train_df = pd.read_csv(pathTrain)

In [5]:
sample_train_df = copy.deepcopy(train_df)
sample_test_df = copy.deepcopy(test_df)

In [6]:
def addBoolCol(train_set):
    hashtags = train_set.text.str.find("#") != -1
    links = train_set.text.str.find("http") != -1
    personTag = train_set.text.str.find("@") != -1
    keywordNull = train_set.keyword.isna()
    #Adding new col
    train_set["contains_hashtags"] = hashtags
    train_set["contains_usertags"] = personTag
    train_set["contains_weblink"] = links
    train_set["keywordNull"] = keywordNull
    return 0

In [7]:
def removeSymbols(train_set):
    symbols = "!\"$%&'*+,-./:;?[\]^_`{|}~#@"
    numbers = "0123456789"
    for symbol in symbols:
        symbol = "\\" + symbol
        train_set['text'] = train_set['text'].str.replace(symbol," ")
        train_set['keyword'] = train_set['keyword'].str.replace(symbol," ")
    for number in numbers:
        train_set['text'] = train_set['text'].str.replace(number,"")
        train_set['keyword'] = train_set['keyword'].str.replace(number," ")
    return 0

In [8]:
def documentToMatrix(train_set):
    #The modified matrix would look something like this :
    # ----------- word features ----------- hashTagCol --- UserTagCol --- LinkCol ----
    #get tag informaiton
    hashTagCol = train_set.contains_hashtags.astype(int).to_numpy()
    userTagCol = train_set.contains_usertags.astype(int).to_numpy()
    linkCol = train_set.contains_weblink.astype(int).to_numpy()
    #reshaping
    hashTagCol = hashTagCol.reshape(len(hashTagCol),1)
    userTagCol = userTagCol.reshape(len(userTagCol),1)
    linkCol = linkCol.reshape(len(linkCol),1)
    #Make matrix
    count_vectorizer = feature_extraction.text.CountVectorizer()
    matrix = count_vectorizer.fit_transform(train_set.text)
    #Add column to modified matrix
    modifiedMatrix = matrix.toarray()
    modifiedMatrix = np.column_stack((modifiedMatrix,hashTagCol))
    modifiedMatrix = np.column_stack((modifiedMatrix,userTagCol))
    modifiedMatrix = np.column_stack((modifiedMatrix,linkCol))
    return {"matrix": matrix,"vectorizer":count_vectorizer, "modifiedMatrix":modifiedMatrix}     

In [33]:
#Training set
addBoolCol(sample_train_df)
removeSymbols(sample_train_df)
matrixDict = documentToMatrix(sample_train_df)
targetVector = sample_train_df.target.to_numpy()

In [34]:
clf = linear_model.RidgeClassifier()
clf = linear_model.RidgeClassifier().fit(matrixDict["modifiedMatrix"], targetVector)

In [35]:
#Testing set
addBoolCol(sample_test_df)
removeSymbols(sample_test_df)
TestmatrixDict = documentToMatrix(sample_test_df)

In [37]:
clf.decision_function(TestmatrixDict["modifiedMatrix"])

ValueError: X has 11902 features per sample; expecting 20916

<h2> Big problem </h2>
<p> The method used above will have different features for training set than testing set. Therefore the method used above is not usefull! </p>